In [1]:
import numpy as np
import pandas as pd
import h5py
import pysam
import os
from modisco.visualization import viz_sequence
from modisco import util
from matplotlib import pyplot as plt
import pybedtools
from bs4 import BeautifulSoup

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [2]:
dttype = 'ATAC'
mode = "counts"
celltype="K562"
modisco_path = '/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/ATAC/'+celltype+'/merge_folds_new/'+mode+'/modisco_old_format.h5'
ppm_dir = ''

htmld="/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/"+dttype+'/'+celltype+"/merge_folds_new/"+mode+"/motifs.html"
tomtom = pd.read_html(htmld)


background=[0.25, 0.25, 0.25, 0.25]

In [3]:
def trim_motif_new(cwm, motif, trim_threshold=0.20):
    """
    Given the PFM and motif (both L x 4 arrays) (the motif could be the
    PFM itself), trims `motif` by cutting off flanks of low information
    content in `pfm`. `min_ic` is the minimum required information
    content. If specified this trimmed motif will be extended on either
    side by `pad` bases.
    If no base passes the `min_ic` threshold, then no trimming is done.
    """
    
    score = np.sum(np.abs(cwm), axis=1)
    trim_thresh = np.max(score) * trim_threshold  # Cut off anything less than 30% of max score
    pass_inds = np.where(score >= trim_thresh)[0]
    trimmed = motif[np.min(pass_inds): np.max(pass_inds) + 1]
 
    if not trimmed.size:
        return motif
    
    return trimmed

def import_tfmodisco_motifs(tfm_results_path, trim=True, only_pos=True):
    """
    Imports the PFMs to into a dictionary, mapping `(x, y)` to the PFM,
    where `x` is the metacluster index and `y` is the pattern index.
    Arguments:
        `tfm_results_path`: path to HDF5 containing TF-MoDISco results
        `out_dir`: where to save motifs
        `trim`: if True, trim the motif flanks based on information content
        `only_pos`: if True, only return motifs with positive contributions
    Returns the dictionary of PFMs.
    """ 
    pfms = {}
    with h5py.File(tfm_results_path, "r") as f:
        metaclusters = f["metacluster_idx_to_submetacluster_results"]
        num_metaclusters = len(metaclusters.keys())
        for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
            metacluster = metaclusters[metacluster_key]
            if "patterns" not in metacluster["seqlets_to_patterns_result"]:
                continue
            patterns = metacluster["seqlets_to_patterns_result"]["patterns"]
            num_patterns = len(patterns["all_pattern_names"][:])
            for pattern_i, pattern_name in enumerate(patterns["all_pattern_names"][:]):
#                pattern_name = pattern_name.decode()
                pattern_name = pattern_name

                pattern = patterns[pattern_name]
                pfm = pattern["sequence"]["fwd"][:]
                cwm = pattern["task0_contrib_scores"]["fwd"][:]
                
                # Check that the contribution scores are overall positive
                if only_pos and np.sum(cwm) < 0:
                    continue
                    
                if trim:
                    pfm = trim_motif_new(cwm, cwm)
                else:
                    pfm = cwm
                    
                pfms["%d_%d" % (metacluster_i,pattern_i)] = pfm
    return pfms

In [4]:
pfms = import_tfmodisco_motifs(modisco_path, trim=False)

In [5]:
for key in pfms:
    f = open(os.path.join(ppm_dir,mode+"_"+key+".pfm"),"w")
    #print(pfms[key])
    np.savetxt(f, pfms[key], fmt='%f')
    f.close()
    

In [6]:
tomtom[0]

,pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
0,pos_patterns.pattern_0,36366,NaN,NaN,GATA3_GATA_1,2.261690e-01,NaN,GATA3_GATA_2,2.261690e-01,NaN,GATA3_MA0037.3,2.261690e-01,NaN
1,pos_patterns.pattern_1,33817,NaN,NaN,CTCF_MA0139.1,1.215460e-15,NaN,CTCF_HUMAN.H11MO.0.A,5.730620e-12,NaN,CTCF_MOUSE.H11MO.0.A,3.063360e-11,NaN
2,pos_patterns.pattern_2,26683,NaN,NaN,KLF12_HUMAN.H11MO.0.C,8.183800e-06,NaN,KLF3_HUMAN.H11MO.0.B,4.967560e-05,NaN,KLF3_MOUSE.H11MO.0.A,4.967560e-05,NaN
3,pos_patterns.pattern_3,13473,NaN,NaN,BACH2_MOUSE.H11MO.0.A,5.835680e-06,NaN,BACH2_HUMAN.H11MO.0.A,5.835680e-06,NaN,NFE2_HUMAN.H11MO.0.A,7.780910e-06,NaN
4,pos_patterns.pattern_4,9415,NaN,NaN,Gabpa_MA0062.2,9.275380e-05,NaN,EHF_ETS_1,9.275380e-05,NaN,EHF_MA0598.2,9.275380e-05,NaN
5,pos_patterns.pattern_5,5999,NaN,NaN,NFYB_HUMAN.H11MO.0.A,1.351140e-03,NaN,NFYB_MOUSE.H11MO.0.A,1.351140e-03,NaN,NFYC_HUMAN.H11MO.0.A,1.351140e-03,NaN
6,pos_patterns.pattern_6,4151,NaN,NaN,CEBPG_HUMAN.H11MO.0.B,3.531970e-04,NaN,ATF4_HUMAN.H11MO.0.A,4.210770e-04,NaN,ATF4_MOUSE.H11MO.0.A,4.210770e-04,NaN
7,pos_patterns.pattern_7,3616,NaN,NaN,ETV4_MOUSE.H11MO.0.B,2.549890e-03,NaN,ELF5_HUMAN.H11MO.0.A,3.759180e-03,NaN,EHF_HUMAN.H11MO.0.B,3.759180e-03,NaN
8,pos_patterns.pattern_8,3234,NaN,NaN,NRF1_MA0506.1,3.172790e-05,NaN,NRF1_MOUSE.H11MO.0.A,3.172790e-05,NaN,NRF1_HUMAN.H11MO.0.A,3.172790e-05,NaN
9,pos_patterns.pattern_9,2722,NaN,NaN,BACH2_HUMAN.H11MO.0.A,1.757660e-04,NaN,BACH2_MOUSE.H11MO.0.A,1.757660e-04,NaN,NF2L2_MOUSE.H11MO.0.A,1.757660e-04,NaN


In [7]:

tomtom[0]["pattern"] = tomtom[0]["pattern"].str.replace("pos_patterns.pattern","0").str.replace("neg_patterns.pattern","1")


/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [8]:
tomtom[0]

,pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
0,0_0,36366,NaN,NaN,GATA3_GATA_1,2.261690e-01,NaN,GATA3_GATA_2,2.261690e-01,NaN,GATA3_MA0037.3,2.261690e-01,NaN
1,0_1,33817,NaN,NaN,CTCF_MA0139.1,1.215460e-15,NaN,CTCF_HUMAN.H11MO.0.A,5.730620e-12,NaN,CTCF_MOUSE.H11MO.0.A,3.063360e-11,NaN
2,0_2,26683,NaN,NaN,KLF12_HUMAN.H11MO.0.C,8.183800e-06,NaN,KLF3_HUMAN.H11MO.0.B,4.967560e-05,NaN,KLF3_MOUSE.H11MO.0.A,4.967560e-05,NaN
3,0_3,13473,NaN,NaN,BACH2_MOUSE.H11MO.0.A,5.835680e-06,NaN,BACH2_HUMAN.H11MO.0.A,5.835680e-06,NaN,NFE2_HUMAN.H11MO.0.A,7.780910e-06,NaN
4,0_4,9415,NaN,NaN,Gabpa_MA0062.2,9.275380e-05,NaN,EHF_ETS_1,9.275380e-05,NaN,EHF_MA0598.2,9.275380e-05,NaN
5,0_5,5999,NaN,NaN,NFYB_HUMAN.H11MO.0.A,1.351140e-03,NaN,NFYB_MOUSE.H11MO.0.A,1.351140e-03,NaN,NFYC_HUMAN.H11MO.0.A,1.351140e-03,NaN
6,0_6,4151,NaN,NaN,CEBPG_HUMAN.H11MO.0.B,3.531970e-04,NaN,ATF4_HUMAN.H11MO.0.A,4.210770e-04,NaN,ATF4_MOUSE.H11MO.0.A,4.210770e-04,NaN
7,0_7,3616,NaN,NaN,ETV4_MOUSE.H11MO.0.B,2.549890e-03,NaN,ELF5_HUMAN.H11MO.0.A,3.759180e-03,NaN,EHF_HUMAN.H11MO.0.B,3.759180e-03,NaN
8,0_8,3234,NaN,NaN,NRF1_MA0506.1,3.172790e-05,NaN,NRF1_MOUSE.H11MO.0.A,3.172790e-05,NaN,NRF1_HUMAN.H11MO.0.A,3.172790e-05,NaN
9,0_9,2722,NaN,NaN,BACH2_HUMAN.H11MO.0.A,1.757660e-04,NaN,BACH2_MOUSE.H11MO.0.A,1.757660e-04,NaN,NF2L2_MOUSE.H11MO.0.A,1.757660e-04,NaN


In [9]:
tomtom[0][["pattern","num_seqlets"]].to_csv(os.path.join(ppm_dir,mode+"_counts.csv"),sep=",",index=False, header=False)